In [3]:
# ==============================================================================
# Step 1: Import Necessary Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from datetime import date, timedelta
import matplotlib.pyplot as plt

# ==============================================================================
# Step 2: Define Model Components (Time2Vec and Transformer Encoder)
# ==============================================================================

class Time2Vec(layers.Layer):
    """Custom Keras layer for Time2Vec embedding."""
    def __init__(self, kernel_size=1, **kwargs):
        super(Time2Vec, self).__init__(**kwargs)
        self.k = kernel_size

    def build(self, input_shape):
        self.wb = self.add_weight(name='wb', shape=(input_shape[-1],), initializer='uniform', trainable=True)
        self.bb = self.add_weight(name='bb', shape=(input_shape[-1],), initializer='uniform', trainable=True)
        self.wa = self.add_weight(name='wa', shape=(1, input_shape[-2], self.k), initializer='uniform', trainable=True)
        self.ba = self.add_weight(name='ba', shape=(1, input_shape[-2], self.k), initializer='uniform', trainable=True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs):
        bias = self.wb * inputs + self.bb
        dp = K.dot(inputs, self.wa) + self.ba
        wgts = K.sin(dp)
        ret = K.concatenate([K.expand_dims(bias, -1), wgts], -1)
        ret = K.reshape(ret, (-1, ret.shape, ret.shape * ret.shape))
        return ret

    def compute_output_shape(self, input_shape):
        return (input_shape, input_shape, input_shape * (self.k + 1))

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    """A single Transformer encoder block."""
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

# ==============================================================================
# Step 3: Define Model Building Function
# ==============================================================================

def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks,
                mlp_units, dropout=0, mlp_dropout=0, time2vec_dim=1):
    """Builds the complete Transformer model."""
    inputs = tf.keras.Input(shape=input_shape)
    time_embedding = Time2Vec(kernel_size=time2vec_dim)(inputs)
    x = layers.Concatenate(axis=-1)([inputs, time_embedding])
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    outputs = layers.Dense(4)(x)  # Predicting 4 features: Open, High, Low, Close
    return tf.keras.Model(inputs, outputs)

# ==============================================================================
# Step 4: Data Acquisition and Preprocessing
# ==============================================================================

def create_dataset(dataset, look_back=60):
    """Creates input sequences (X) and corresponding target values (y)."""
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), :])
        y.append(dataset[i + look_back, :])
    return np.array(X), np.array(y)

# --- Configuration ---
TICKER_SYMBOL = "NVDA"
LOOKBACK_WINDOW = 60
START_DATE = (date.today() - timedelta(days=5*365)).strftime('%Y-%m-%d')
END_DATE = date.today().strftime('%Y-%m-%d')

# --- Data Loading ---
stock_data = yf.download(TICKER_SYMBOL, start=START_DATE, end=END_DATE)
ohlc_data = stock_data[['Open', 'High', 'Low', 'Close']]

# --- Normalization and Splitting ---
scaler = MinMaxScaler(feature_range=(0, 1))
split_fraction = 0.9
split_index = int(len(ohlc_data) * split_fraction)
train_data = ohlc_data[:split_index]
test_data = ohlc_data[split_index:]

scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

# --- Create Windowed Datasets ---
X_train, y_train = create_dataset(scaled_train_data, LOOKBACK_WINDOW)
X_test, y_test = create_dataset(scaled_test_data, LOOKBACK_WINDOW)

# ==============================================================================
# Step 5: Model Training
# ==============================================================================

# --- Build Model ---
model = build_model(
    input_shape=(LOOKBACK_WINDOW, X_train.shape),
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=,
    dropout=0.25,
    mlp_dropout=0.4,
    time2vec_dim=64
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mean_squared_error')
model.summary()

# --- Callbacks ---
checkpoint_path = "best_model.weights.h5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# --- Train ---
history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=64,
    validation_split=0.1,
    callbacks=[model_checkpoint, early_stopping],
    verbose=1
)

# ==============================================================================
# Step 6: Evaluation and Visualization
# ==============================================================================

# --- Load Best Weights ---
model.load_weights(checkpoint_path)

# --- Make Predictions on Test Set ---
test_predictions_scaled = model.predict(X_test)
test_predictions = scaler.inverse_transform(test_predictions_scaled)

# --- Plot Results ---
predicted_close_prices = test_predictions[:, 3]
actual_close_prices = test_data['Close'].values
prediction_dates = test_data.index

plt.figure(figsize=(15, 7))
plt.plot(prediction_dates, actual_close_prices, color='blue', label='Actual Close Price')
plt.plot(prediction_dates, predicted_close_prices, color='red', linestyle='--', label='Predicted Close Price')
plt.title(f'{TICKER_SYMBOL} Stock Price Prediction (Test Set)')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

# --- Performance Metrics ---
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(actual_close_prices, predicted_close_prices)
mae = mean_absolute_error(actual_close_prices, predicted_close_prices)
print(f"\n--- Model Performance on Test Set ---")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# ==============================================================================
# Step 7: Predict the Next Day
# ==============================================================================

# --- Prepare Input for Next-Day Prediction ---
# Use the last LOOKBACK_WINDOW days from the entire available dataset
full_scaled_data = scaler.transform(ohlc_data)
last_sequence = full_scaled_data
last_sequence = np.expand_dims(last_sequence, axis=0)

# --- Predict and Inverse Transform ---
predicted_scaled_ohlc = model.predict(last_sequence)
predicted_ohlc = scaler.inverse_transform(predicted_scaled_ohlc)
next_day_date = ohlc_data.index[-1] + timedelta(days=1)

print(f"\n--- Forecast for {next_day_date.strftime('%Y-%m-%d')} ---")
print(f"Predicted Open:  ${predicted_ohlc:.2f}")
print(f"Predicted High:  ${predicted_ohlc:.2f}")
print(f"Predicted Low:   ${predicted_ohlc:.2f}")
print(f"Predicted Close: ${predicted_ohlc:.2f}")

SyntaxError: invalid syntax (1198353847.py, line 126)